In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from sklearn.neighbors import NearestNeighbors
from termcolor import colored
from collections import Counter
import json

pd.set_option('display.max_colwidth', None)

In [2]:
inspections = pd.read_csv("prohlidky.csv")
inspections = inspections.drop(['Unnamed: 0'], axis=1)
inspections.DatKont = pd.to_datetime(inspections.DatKont)
inspections.DatPrvReg = pd.to_datetime(inspections.DatPrvReg)

In [ ]:
#failed attempt to use kNN

# tmp_df_copy = tmp_df[['index', 'timestamp_int']]
# sample = tmp_df_copy.sample()
# neighbours = neigh.kneighbors(sample, 5, return_distance=False)

# for neighbour in neighbours:
#     for neighb in neighbour:
#         display(tmp_df.iloc[[neighb]].DatKont)
        
# tmp_df.where(tmp_df.DatKont.dt.month == 7).dropna(how='all').where(tmp_df.DatKont.dt.day == 23).dropna(how='all').where(tmp_df.DatKont.dt.hour == 11).dropna(how='all')

In [4]:
class VehicleModel:
    def __init__(self, name, count, neighbours):
        self.name = name
        self.count = count
        self.neighbours = neighbours
        self.neighbours_count = 0

In [50]:
tmp_df = inspections.where(inspections.STK == 3307).dropna(how='all').sort_values(by=['DatKont']).reset_index()

In [51]:
display(len(tmp_df.drop_duplicates(['TZn', 'ObchOznTyp']).index))
display(len(tmp_df.drop_duplicates(['TZn']).index))

6105

720

In [52]:
vehicles = {}

for TZn in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
    neighbours = {}
    
    for TZnN in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
        neighbours.update({TZnN: 0})
    
    model = VehicleModel(TZn, len(tmp_df[tmp_df.TZn == TZn].index), neighbours)
    vehicles.update({TZn: model})

In [53]:
#27 min - average inspection duration 

for index, row in tmp_df.iterrows():
    for i, neighbour in tmp_df[(tmp_df.DatKont < (row.DatKont + timedelta(minutes=27))) & (tmp_df.DatKont > (row.DatKont - timedelta(minutes=27)))].iterrows():
        if neighbour.TZn != row.TZn:
            vehicles[row.TZn].neighbours[neighbour.TZn] += 1
            vehicles[row.TZn].neighbours_count +=1

In [54]:
#ideally should not print anything - just cheking if it went well or not

for vehicle1 in vehicles.keys():
    for vehicle2 in vehicles.keys():
        if vehicles[vehicle1].neighbours[vehicle2] != vehicles[vehicle2].neighbours[vehicle1]:
            print('an error occured')

In [ ]:
#stations with suspicious behaviour detected
suspicious = []
inspections = inspections[inspections.DrVoz == 'OSOBNI AUTOMOBIL']

for station in inspections.STK.unique():
    tmp_df = inspections.where(inspections.STK == station).dropna(how='all').sort_values(by=['DatKont']).reset_index()

    vehicles = {}

    for TZn in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
        neighbours = {}

        for TZnN in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
            neighbours.update({TZnN: 0})

        model = VehicleModel(TZn, len(tmp_df[tmp_df.TZn == TZn].index), neighbours)
        vehicles.update({TZn: model})

    for index, row in tmp_df.iterrows():
        for i, neighbour in tmp_df[(tmp_df.DatKont < (row.DatKont + timedelta(minutes=27))) & (tmp_df.DatKont > (row.DatKont - timedelta(minutes=27)))].iterrows():
            if neighbour.TZn != row.TZn:
                vehicles[row.TZn].neighbours[neighbour.TZn] += 1
                vehicles[row.TZn].neighbours_count +=1

    total = len(tmp_df.index)

    for vehicle1 in vehicles.keys():
        for vehicle2 in vehicles.keys():
            if vehicle1 != vehicle2 and vehicles[vehicle1].neighbours[vehicle2] > 1:     

                naive_probability = vehicles[vehicle2].count/total
                estimated = naive_probability*vehicles[vehicle1].neighbours_count
                actual = vehicles[vehicle1].neighbours[vehicle2]
                difference = round(100*abs(naive_probability*vehicles[vehicle1].neighbours_count-vehicles[vehicle1].neighbours[vehicle2])/((naive_probability*vehicles[vehicle1].neighbours_count+vehicles[vehicle1].neighbours[vehicle2])/2))


                if difference > 100 and actual > 10:
                    suspicious.append(station)
    
    print(station)

In [58]:
#number of stations with suspicious behaviour detected
len(suspicious)

46

In [ ]:
str(station_conjunctions_infos[3848].suspicious_sorted)

In [ ]:
inspections = inspections[inspections.DrVoz == 'OSOBNI AUTOMOBIL']
inspections.info()

In [ ]:
#final script for all of the stations

class VehicleModel:
    def __init__(self, name, count, neighbours):
        self.name = name
        self.count = count
        self.neighbours = neighbours
        self.neighbours_count = 0

vehicles = {}
vehicle_conjunctions_infos = {}

brands = inspections.TZn.unique()

for TZn in brands:
    neighbours = {}

    for TZnN in brands:
        neighbours.update({TZnN: 0})

    model = VehicleModel(TZn, len(inspections[inspections.TZn == TZn].index), neighbours)
    vehicles.update({TZn: model})
    
print('Template created.')

#27 min - average inspection duration

finished = 0

for index, row in inspections.iterrows():     
    tmp_df = inspections[inspections.STK == row.STK][(inspections.DatKont < (row.DatKont + timedelta(minutes=27))) & (inspections.DatKont > (row.DatKont - timedelta(minutes=27)))]
    for i, neighbour in tmp_df.iterrows():
        if neighbour.VIN != row.VIN: 
            vehicles[row.TZn].neighbours[neighbour.TZn] += 1
            vehicles[row.TZn].neighbours_count +=1
    finished += 1
    print('Finished: {0}'.format(finished))
            
print('Neighbours found.')

In [42]:
class VehicleConjunctionsInfo:
    def __init__(self, vehicle1, vehicle2, estimated, actual, prob, diff):
        self.vehicle1 = vehicle1
        self.vehicle2 = vehicle2
        self.estimated = estimated
        self.actual = actual
        self.prob = prob
        self.diff = diff
        
total = len(inspections.index)

suspicious = {}

for vehicle1 in vehicles.keys():
    for vehicle2 in vehicles.keys():
        if vehicle1 != vehicle2 and vehicles[vehicle1].neighbours[vehicle2] > 1:     

            naive_probability = vehicles[vehicle2].count/total
            estimated = naive_probability*vehicles[vehicle1].neighbours_count
            actual = vehicles[vehicle1].neighbours[vehicle2]
            difference = round(100*abs(naive_probability*vehicles[vehicle1].neighbours_count-vehicles[vehicle1].neighbours[vehicle2])/((naive_probability*vehicles[vehicle1].neighbours_count+vehicles[vehicle1].neighbours[vehicle2])/2))

            con = VehicleConjunctionsInfo(vehicle1, vehicle2, estimated, actual, naive_probability, difference)
            suspicious.update({'{0} & {1}'.format(vehicle1, vehicle2) : con})
    print('Vehicle', vehicle1, 'finished.')

Vehicle MAZDA finished.
Vehicle PEUGEOT finished.
Vehicle SKODA finished.
Vehicle FORD finished.
Vehicle LEXUS finished.
Vehicle VW finished.
Vehicle OPEL finished.
Vehicle CITROEN finished.
Vehicle MERCEDES-BENZ finished.
Vehicle RENAULT finished.
Vehicle VOLKSWAGEN finished.
Vehicle AUDI finished.
Vehicle SUBARU finished.
Vehicle KIA finished.
Vehicle FIAT finished.
Vehicle BMW finished.
Vehicle HYUNDAI finished.
Vehicle HONDA finished.
Vehicle ALFA ROMEO finished.
Vehicle TOYOTA finished.
Vehicle KAWASAKI finished.
Vehicle CHRYSLER finished.
Vehicle JEEP finished.
Vehicle VOLVO finished.
Vehicle PORSCHE finished.
Vehicle CHEVROLET finished.
Vehicle DACIA finished.
Vehicle MERCEDES finished.
Vehicle DAEWOO finished.
Vehicle SEAT finished.
Vehicle LAND ROVER finished.
Vehicle NISSAN finished.
Vehicle SUZUKI finished.
Vehicle MITSUBISHI finished.
Vehicle JAGUAR finished.
Vehicle DETHLEFFS finished.
Vehicle VAZ finished.
Vehicle MINI finished.
Vehicle MASERATI finished.
Vehicle SAAB fin

In [43]:
suspicious_df = pd.DataFrame(columns=['pair', 'vehicle1', 'vehicle2', 'estimated', 'actual', 'naive_probability', 'difference'])
i = 0 
for pair in suspicious.keys():
    suspicious_df.loc[i] = [pair] + [suspicious[pair].vehicle1] + [suspicious[pair].vehicle2] + [suspicious[pair].estimated] + [suspicious[pair].actual] + [suspicious[pair].prob] + [suspicious[pair].diff]
    i += 1

In [44]:
suspicious_df.sort_values('actual', ascending=False).head(20)

,pair,vehicle1,vehicle2,estimated,actual,naive_probability,difference
622,FORD & SKODA,FORD,SKODA,292924.680334,278714,0.317279,5
295,SKODA & FORD,SKODA,FORD,280955.829388,278714,0.079680,1
887,VW & SKODA,VW,SKODA,212316.225841,202986,0.317279,4
297,SKODA & VW,SKODA,VW,190227.916197,202986,0.053950,6
294,SKODA & PEUGEOT,SKODA,PEUGEOT,202544.034572,196672,0.057442,3
110,PEUGEOT & SKODA,PEUGEOT,SKODA,202372.077638,196672,0.317279,3
1509,RENAULT & SKODA,RENAULT,SKODA,199593.351010,192941,0.317279,3
301,SKODA & RENAULT,SKODA,RENAULT,202650.915904,192941,0.057473,5
1683,VOLKSWAGEN & SKODA,VOLKSWAGEN,SKODA,181990.412894,177153,0.317279,3
302,SKODA & VOLKSWAGEN,SKODA,VOLKSWAGEN,188108.537588,177153,0.053348,6


In [45]:
suspicious_df = suspicious_df.dropna()

In [63]:
suspicious_df[suspicious_df.actual>suspicious_df.estimated][suspicious_df.actual>10].sort_values('difference', ascending=False).head(15)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,pair,vehicle1,vehicle2,estimated,actual,naive_probability,difference
4738,MERCEDES BENZ & LEXUS,MERCEDES BENZ,LEXUS,0.496785,12,0.000658,184
874,LEXUS & MERCEDES BENZ,LEXUS,MERCEDES BENZ,0.557361,12,0.000061,182
3282,PORSCHE & INFINITI,PORSCHE,INFINITI,2.601227,12,0.000152,129
4526,INFINITI & PORSCHE,INFINITI,PORSCHE,2.687958,12,0.001232,127
4742,MERCEDES BENZ & MERCEDES-BENZ,MERCEDES BENZ,MERCEDES-BENZ,16.501346,71,0.021856,125
4328,MINI & FERRARI,MINI,FERRARI,2.929098,12,0.000135,122
1415,MERCEDES-BENZ & MERCEDES BENZ,MERCEDES-BENZ,MERCEDES BENZ,17.280622,71,0.000061,122
3229,VOLVO & BMW I,VOLVO,BMW I,3.795482,14,0.000028,115
4783,MERCEDES - BENZ & VOLVO,MERCEDES - BENZ,VOLVO,6.554911,23,0.010676,111
3172,VOLVO & MERCEDES - BENZ,VOLVO,MERCEDES - BENZ,6.592154,23,0.000049,111


In [ ]:
inspections[inspections.TZn == 'DAF']

In [ ]:
df = inspections[inspections.STK == 3818][inspections.DatKont.dt.date == pd.to_datetime('2018-05-16')]
df[df.DatKont.dt.hour < 12].sort_values('DatKont')

In [ ]:
df = inspections[inspections.STK == 3818][inspections.DatKont.dt.date == pd.to_datetime('2018-03-28')]
df[df.DatKont.dt.hour < 12].sort_values('DatKont')

In [ ]:
df = inspections[inspections.STK == 3100][inspections.DatKont.dt.date == pd.to_datetime('2018-02-28')]
df[df.DatKont.dt.hour < 13].sort_values('DatKont')

In [58]:
inspections[inspections.TZn == 'FERRARI'].groupby('VyslSTK').count()

,STK,DrTP,VIN,DatKont,TypMot,TZn,DrVoz,ObchOznTyp,Ct,DatPrvReg,Km,ZavA,ZavB,ZavC,VyslEmise
VyslSTK,,,,,,,,,,,,,,,
castecne zpusobile,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
zpusobile,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352


In [49]:
inspections[inspections.TZn == 'SKODA'].groupby('STK').count().sort_values('DrTP')

,DrTP,VIN,DatKont,TypMot,TZn,DrVoz,ObchOznTyp,Ct,DatPrvReg,Km,ZavA,ZavB,ZavC,VyslSTK,VyslEmise
STK,,,,,,,,,,,,,,,
8350,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
8708,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8201,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8203,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8211,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,7568,7568,7568,7564,7568,7568,7568,7568,7568,7568,7568,7568,7568,7568,7568
3307,9217,9217,9217,9200,9217,9217,9217,9217,9217,9217,9217,9217,9217,9216,9217
3234,9585,9585,9585,9582,9585,9585,9585,9585,9585,9585,9585,9585,9585,9585,9585


In [57]:
inspections[inspections.STK == 3122][inspections.TZn == 'FERRARI'].groupby('VyslSTK').count()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,STK,DrTP,VIN,DatKont,TypMot,TZn,DrVoz,ObchOznTyp,Ct,DatPrvReg,Km,ZavA,ZavB,ZavC,VyslEmise
VyslSTK,,,,,,,,,,,,,,,
castecne zpusobile,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
zpusobile,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54


In [56]:
inspections[inspections.STK == 3122][inspections.DatKont.dt.date == pd.to_datetime('2018-05-16')].sort_values('DatKont').head(30)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,STK,DrTP,VIN,DatKont,TypMot,TZn,DrVoz,ObchOznTyp,Ct,DatPrvReg,Km,ZavA,ZavB,ZavC,VyslSTK,VyslEmise
1790596,3122,pravidelna,WF0GXXGBBGAT75253,2018-05-16 07:57:35.007,UFBA,FORD,OSOBNI AUTOMOBIL,MONDEO (BA7),M1,2010-04-22,111053,0,0,0,zpusobile,vyhovuje
1260965,3122,pravidelna,WBAEL91010PA25691,2018-05-16 08:05:24.313,306D1,BMW,OSOBNI AUTOMOBIL,330 (346L),M1,2001-11-06,289544,4,0,0,zpusobile,vyhovuje
2886812,3122,Evidencni kontrola,WF03XXGCD35U67841,2018-05-16 08:06:15.513,HWDA,FORD,OSOBNI AUTOMOBIL,FOCUS (DA3),M1,2005-03-24,151425,0,0,0,zpusobile,---
2729913,3122,Evidencni kontrola,WDD2052071F564843,2018-05-16 08:16:12.520,651921,MERCEDES-BENZ,OSOBNI AUTOMOBIL,C 200,M1,2017-04-12,16265,0,0,0,zpusobile,---
2443562,3122,Evidencni kontrola,WDD1760121J593545,2018-05-16 08:20:03.977,K9K,MERCEDES-BENZ,OSOBNI AUTOMOBIL,A 180,M1,2017-03-20,28713,0,0,0,zpusobile,---
851506,3122,pravidelna,W0LFXB32S8M141733,2018-05-16 08:22:04.220,K12B,OPEL,OSOBNI AUTOMOBIL,AGILA,M1,2008-07-14,43961,0,0,0,zpusobile,vyhovuje
421586,3122,Evidencni kontrola,WDD2052071F565157,2018-05-16 08:26:15.800,651921,MERCEDES-BENZ,OSOBNI AUTOMOBIL,C 200,M1,2017-04-12,19954,0,0,0,zpusobile,---
1203091,3122,pravidelna,VF77J9HL0EJ657088,2018-05-16 08:32:06.593,9H05,CITROEN,OSOBNI AUTOMOBIL,BERLINGO,M1,2014-05-21,34446,0,0,0,zpusobile,vyhovuje
1250924,3122,pravidelna,WVWZZZ1JZWW181311,2018-05-16 08:36:30.327,AGR,VOLKSWAGEN,OSOBNI AUTOMOBIL,GOLF,M1,1999-09-15,209216,2,0,0,zpusobile,vyhovuje
939149,3122,Evidencni kontrola,WBA2G11050V637393,2018-05-16 09:00:23.647,B58B30A,BMW,OSOBNI AUTOMOBIL,M240i,M1,2017-02-13,8667,0,0,0,zpusobile,---
